In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import xgboost
import lightgbm as lgb

In [3]:
pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 626.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=c8e628f24d9ceddb9faa247b14d4c915b0fdd7169eaa1a0e32a2514611e80424
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
from tpot import TPOTClassifier

In [5]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [7]:
import os
path="/content/drive/My Drive/Colab Notebooks/"
os.chdir(path)
os.listdir(path)

['“FirstNeuralNetworkSolution - START HERE.ipynb”的副本',
 'ecuador-store-sales-global-forecasting-lightgbm.ipynb',
 'Untitled (1)',
 'Untitled6.ipynb',
 'data_proc.ipynb',
 '“dapr.ipynb”的副本',
 '“stable_diffusion.ipynb”的副本',
 'Blockchain.csv',
 '机器学习导论第二课-聚类.ipynb',
 'Untitled8.ipynb',
 'EMB-SVR比特币预测.ipynb',
 'DeepAR-pytorch-master',
 'prophet-main',
 'Untitled9.ipynb',
 'mictools-master',
 'alphasickle-master',
 'Untitled0.ipynb',
 'Untitled1.ipynb',
 'Untitled2.ipynb',
 'EMD_LSTM-master',
 'Untitled3.ipynb',
 'EMD-LSTM-MLP-master',
 'Untitled4.ipynb',
 'FinGPT',
 '000985.csv',
 'dataset',
 'train.csv',
 'test.csv',
 'Untitled',
 'new_stockdata',
 '聚宽笔试2.ipynb',
 'y_pre_test.csv',
 '运行py文件.ipynb',
 '9th-kaggle-optiver-trading-close-main',
 '金融诈骗lasso+tpot.ipynb（2）',
 '金融诈骗lasso+tpot.ipynb']

In [30]:
# 读取数据
data = pd.read_csv('train.csv')
data_target = data['fraud']
data = data.drop(data.columns[-1], axis=1)


In [31]:
# 将年龄分箱
bins = [0, 18, 30, 45, 60, 120]
labels = ['child', 'young', 'adult', 'middle_aged', 'senior']
data['age_bin'] = pd.cut(data['age'], bins=bins, labels=labels)
# data = pd.get_dummies(data, columns=['age_bin'])

# 提取policy_bind_date和incident_date的年月日
data['policy_year'] = pd.to_datetime(data['policy_bind_date']).dt.year
data['policy_month'] = pd.to_datetime(data['policy_bind_date']).dt.month
data['policy_day'] = pd.to_datetime(data['policy_bind_date']).dt.day
data['incident_year'] = pd.to_datetime(data['incident_date']).dt.year
data['incident_month'] = pd.to_datetime(data['incident_date']).dt.month
data['incident_day'] = pd.to_datetime(data['incident_date']).dt.day

# 计算policy_bind_date到incident_date的时间差
data['days_to_incident'] = (pd.to_datetime(data['incident_date']) - pd.to_datetime(data['policy_bind_date'])).dt.days

# 对类别特征进行One-Hot编码
cat_cols = ['policy_state', 'insured_sex', 'insured_education_level', 'insured_occupation',
            'insured_hobbies', 'insured_relationship', 'incident_type', 'collision_type',
            'authorities_contacted', 'incident_state', 'incident_city', 'auto_make', 'auto_model','incident_severity']
data = pd.get_dummies(data, columns=cat_cols)

# 计算incident_hour_of_the_day的sin和cos值
data['incident_hour_sin'] = np.sin(2 * np.pi * data['incident_hour_of_the_day'] / 24)
data['incident_hour_cos'] = np.cos(2 * np.pi * data['incident_hour_of_the_day'] / 24)

# 计算资本收益和损失的比例
data['capital_gain_loss_ratio'] = data['capital-gains'] / (data['capital-loss'] + 1)

# 填充缺失值
data['police_report_available'] = data['police_report_available'].fillna(data['police_report_available'].mode()[0])
data['property_damage'] = data['property_damage'].fillna(data['property_damage'].mode()[0])

# 计算索赔金额的总和和比例
data['total_claim_ratio'] = data['total_claim_amount'] / (data['injury_claim'] + data['property_claim'] + data['vehicle_claim'] + 1)
data['claim_sum'] = data['injury_claim'] + data['property_claim'] + data['vehicle_claim']

# 对于高度偏态的数值特征进行对数转换
skewed_features = ['capital-gains', 'capital-loss', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim']
for feat in skewed_features:
    data[feat] = np.log1p(data[feat])

# 处理日期特征
data['policy_bind_date'] = data['policy_bind_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data['incident_date'] = data['incident_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data['auto_year'] = data['auto_year'].apply(lambda x: int(pd.to_datetime(x).timestamp()))

data['property_damage'] = data['property_damage'].replace('?', pd.NA)
data['police_report_available'] = data['police_report_available'].replace('?', pd.NA)

data['property_damage'] = data['property_damage'].fillna(0)
data['police_report_available'] = data['police_report_available'].fillna(0)

# 定义一个函数来转换布尔型字符串为整数
def str_to_int(val):
    if val == 'YES':
        return 1
    elif val == 'NO':
        return -1
    else:
        return val


# 遍历每一列，如果列中包含'yes'或'no'，则应用转换函数
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].apply(str_to_int)

# 迭代遍历所有列
for column in data.columns:
    # 如果列的数据类型是布尔类型
    if data[column].dtype == bool:
        # 将布尔类型的列转换为整数类型，并将 True 转换为 1，False 转换为 -1
        data[column] = data[column].astype(int)
        data[column] = data[column].replace({0: -1})

data = data.drop(columns=['policy_id','age_bin','policy_csl'])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [32]:
X_train = data

In [33]:
y_train = data_target

In [34]:
from xgboost import XGBClassifier
import lightgbm as lgb

# XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
feature_importances_xgb = xgb_model.feature_importances_
feature_importance_df_xgb = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances_xgb})
feature_importance_df_xgb = feature_importance_df_xgb.sort_values('importance', ascending=False)

# 选择重要性前20%的特征作为强特征
num_top_features = int(0.2 * len(X_train.columns))
top_features_xgb = feature_importance_df_xgb.head(num_top_features)['feature'].tolist()

# LightGBM
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)
feature_importances_lgb = lgb_model.feature_importances_
feature_importance_df_lgb = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances_lgb})
feature_importance_df_lgb = feature_importance_df_lgb.sort_values('importance', ascending=False)

# 选择重要性前20%的特征作为强特征
num_top_features = int(0.2 * len(X_train.columns))
top_features_lgb = feature_importance_df_lgb.head(num_top_features)['feature'].tolist()

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 181, number of negative: 519
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001122 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3472
[LightGBM] [Info] Number of data points in the train set: 700, number of used features: 142
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258571 -> initscore=-1.053407
[LightGBM] [Info] Start training from score -1.053407
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [15]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel

In [16]:
# 分离特征和目标变量
# 填充X中的NaN值
imputer = SimpleImputer(strategy='mean')
data = imputer.fit_transform(data)

In [38]:
y = data_target

In [ ]:
lasso = LassoCV(cv=5, random_state=42, alphas=[0.001, 0.01, 0.1, 1, 10, 100])
lasso.fit(data, y)
feature_mask = SelectFromModel(lasso, prefit=True)
data = feature_mask.transform(data)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.899267116573498, tolerance: 0.010841250000000005
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29.487711245173468, tolerance: 0.010982142857142855
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.27303283409736, tolerance: 0.011074285714285712
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:664: Con

In [18]:
data =  pd.DataFrame(data)

In [28]:
top_features_lgb

['policy_annual_premium',
 'incident_date',
 'insured_zip',
 'total_claim_ratio',
 'vehicle_claim',
 'property_claim',
 'customer_months',
 'incident_severity_Major Damage',
 'age',
 'total_claim_amount',
 'injury_claim',
 'policy_bind_date',
 'incident_day',
 'policy_month',
 'days_to_incident',
 'policy_day',
 'incident_hour_cos',
 'witnesses',
 'incident_hour_sin',
 'incident_hour_of_the_day',
 'insured_hobbies_chess',
 'capital-gains',
 'collision_type_Rear Collision',
 'claim_sum',
 'insured_hobbies_cross-fit',
 'capital_gain_loss_ratio',
 'police_report_available',
 'number_of_vehicles_involved',
 'incident_month',
 'bodily_injuries',
 'insured_hobbies_camping',
 'authorities_contacted_Ambulance',
 'policy_deductable']

In [48]:
selected_data = data[top_features_lgb]

In [49]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import PolynomialFeatures
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

In [ ]:
# # Average CV score on the training set was: 0.9799428471757372
# exported_pipeline = make_pipeline(
#     PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
#     StackingEstimator(estimator=LogisticRegression(C=0.1, dual=False, penalty="l2")),
#     RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.35000000000000003, min_samples_leaf=20, min_samples_split=19, n_estimators=100)
# )
# # Fix random state for all the steps in exported pipeline
# set_param_recursive(exported_pipeline.steps, 'random_state', 42)

# exported_pipeline.fit(data, y)
# results = exported_pipeline.predict(data)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# print(exported_pipeline.score(data, y))

0.8871428571428571


In [58]:
selected_data.shape

(700, 33)

In [50]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2)
tpot.fit(selected_data, y)
print(tpot.score(selected_data, y))

Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


2.01 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.6500000000000001, min_samples_leaf=6, min_samples_split=10, n_estimators=100)
0.95


In [ ]:
y_pre = tpot.predict(data)

In [51]:
# 读取数据
data_test = pd.read_csv('test.csv')
# 将年龄分箱
bins = [0, 18, 30, 45, 60, 120]
labels = ['child', 'young', 'adult', 'middle_aged', 'senior']
data_test['age_bin'] = pd.cut(data_test['age'], bins=bins, labels=labels)
# 提取policy_bind_date和incident_date的年月日
data_test['policy_year'] = pd.to_datetime(data_test['policy_bind_date']).dt.year
data_test['policy_month'] = pd.to_datetime(data_test['policy_bind_date']).dt.month
data_test['policy_day'] = pd.to_datetime(data_test['policy_bind_date']).dt.day
data_test['incident_year'] = pd.to_datetime(data_test['incident_date']).dt.year
data_test['incident_month'] = pd.to_datetime(data_test['incident_date']).dt.month
data_test['incident_day'] = pd.to_datetime(data_test['incident_date']).dt.day

# 计算policy_bind_date到incident_date的时间差
data_test['days_to_incident'] = (pd.to_datetime(data_test['incident_date']) - pd.to_datetime(data_test['policy_bind_date'])).dt.days
# 对类别特征进行One-Hot编码
cat_cols = ['policy_state', 'insured_sex', 'insured_education_level', 'insured_occupation',
            'insured_hobbies', 'insured_relationship', 'incident_type', 'collision_type',
            'authorities_contacted', 'incident_state', 'incident_city', 'auto_make', 'auto_model','incident_severity']
data_test = pd.get_dummies(data_test, columns=cat_cols)
# 计算incident_hour_of_the_day的sin和cos值
data_test['incident_hour_sin'] = np.sin(2 * np.pi * data_test['incident_hour_of_the_day'] / 24)
data_test['incident_hour_cos'] = np.cos(2 * np.pi * data_test['incident_hour_of_the_day'] / 24)

# 计算资本收益和损失的比例
data_test['capital_gain_loss_ratio'] = data_test['capital-gains'] / (data_test['capital-loss'] + 1)

# 填充缺失值
data_test['police_report_available'] = data_test['police_report_available'].fillna(data_test['police_report_available'].mode()[0])
data_test['property_damage'] = data_test['property_damage'].fillna(data_test['property_damage'].mode()[0])
# 计算索赔金额的总和和比例
data_test['total_claim_ratio'] = data_test['total_claim_amount'] / (data_test['injury_claim'] + data_test['property_claim'] + data_test['vehicle_claim'] + 1)
data_test['claim_sum'] = data_test['injury_claim'] + data_test['property_claim'] + data_test['vehicle_claim']

# 对于高度偏态的数值特征进行对数转换
skewed_features = ['capital-gains', 'capital-loss', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim']
for feat in skewed_features:
    data_test[feat] = np.log1p(data_test[feat])
    # 处理日期特征
data_test['policy_bind_date'] = data_test['policy_bind_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data_test['incident_date'] = data_test['incident_date'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data_test['auto_year'] = data_test['auto_year'].apply(lambda x: int(pd.to_datetime(x).timestamp()))
data_test['property_damage'] = data_test['property_damage'].replace('?', pd.NA)
data_test['police_report_available'] = data_test['police_report_available'].replace('?', pd.NA)
data_test['property_damage'] = data_test['property_damage'].fillna(0)
data_test['police_report_available'] = data_test['police_report_available'].fillna(0)

# 定义一个函数来转换布尔型字符串为整数
def str_to_int(val):
    if val == 'YES':
        return 1
    elif val == 'NO':
        return -1
    else:
        return val

# 遍历每一列，如果列中包含'yes'或'no'，则应用转换函数
for column in data_test.columns:
    if data_test[column].dtype == 'object':
        data_test[column] = data_test[column].apply(str_to_int)
        # 迭代遍历所有列
for column in data_test.columns:
    # 如果列的数据类型是布尔类型
    if data_test[column].dtype == bool:
        # 将布尔类型的列转换为整数类型，并将 True 转换为 1，False 转换为 -1
        data_test[column] = data_test[column].astype(int)
        data_test[column] = data_test[column].replace({0: -1})

# 定义一个函数来转换布尔型字符串为整数
def str_to_int1(val):
    if val == 'YES':
        return 1
    elif val == 'NO':
        return -1
    else:
        return val

# 遍历每一列，如果列中包含'yes'或'no'，则应用转换函数
for column in data_test.columns:
    if data_test[column].dtype == 'object':
        data_test[column] = data_test[column].apply(str_to_int1)
# 定义一个函数来转换布尔型字符串为整数

def str_to_int2(val):
    if val == 'TRUE':
        return 1
    elif val == 'FALSE':
        return -1
    else:
        return val

# 遍历每一列，如果列中包含'yes'或'no'，则应用转换函数
for column in data_test.columns:
    if data_test[column].dtype == 'object':
        data_test[column] = data_test[column].apply(str_to_int2)

data_test= data_test.replace('?', pd.NA)

data_test = data_test.drop(columns=['policy_id','age_bin','policy_csl'])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
imputer_test = SimpleImputer(strategy='mean')
data_test = imputer.fit_transform(data_test)

In [52]:
data_test = pd.DataFrame(data_test)

In [ ]:
data_test.columns = [str(i) for i in range(1, len(data_test.columns) + 1)]
selected_columns = ['1', '2', '4', '5', '8', '11', '13', '15', '19', '23', '24', '26', '27', '30', '32', '34', '44', '59', '60', '61', '75', '78', '87', '88', '89', '91', '94', '95', '131', '157', '160']
data_test = data_test[selected_columns]

In [53]:
selected_data_test = data_test[top_features_lgb]

In [54]:
y_pre_test = tpot.predict(selected_data_test)

In [56]:
y_pre_test = pd.DataFrame(y_pre_test)

In [57]:
y_pre_test.to_csv('y_pre_test_new.csv', index=False)